# Textbook Chatbot

The Textbook Chatbot project for CSE 6550 is designed to assist with queries related to the textbook, "Guide to the Software Engineering Body of Knowledge".

## Table of Contents
1. [Setup](#1.-Setup)
2. [Inference](#2.-Inference)

## 1. Setup

Import required libraries and modules

In [40]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
from ipywidgets import widgets
from backend.inference import chat_completion

Setup question input widget

In [47]:
question_widget = widgets.Text(
    value='',
    description='Enter your question:',
    placeholder='Type your question here...',
    style={'description_width': 'initial'},
    layout={'width': '80%'}
)

## 2. Inference

In [51]:
# Display the input widget
display(question_widget)
user_question = question_widget.value
if not user_question:
    user_question = "Who is Hironori Washizaki?"

Text(value='Hello', description='Enter your question:', layout=Layout(width='80%'), placeholder='Type your que…

In [53]:
for chunk, model_name in chat_completion(user_question):
    print(chunk, end='')

Running prompt: Hello
Hello! It seems like you didn't ask a specific question based on the provided context. The context you provided is about the Guide to the Software Engineering Body of Knowledge (SWEBOK Guide), which is a document published by the IEEE Computer Society that represents the current state of generally accepted knowledge in software engineering. The guide covers various topics related to software engineering, including software testing, software construction, mathematical foundations, and more. If you have a specific question about any of these topics, feel free to ask! I'm here to help.
Sources:  <a href="/team3/?view=pdf&file=/app/data/swebok/textbook.pdf&page=413" target="_blank">[413]</a><a href="/team3/?view=pdf&file=/app/data/swebok/textbook.pdf&page=148" target="_blank">[148]</a><a href="/team3/?view=pdf&file=/app/data/swebok/textbook.pdf&page=137" target="_blank">[137]</a><a href="/team3/?view=pdf&file=/app/data/swebok/textbook.pdf&page=138" target="_blank">[13